# HPO ontology loading

In [2]:
import os
import time 
import tqdm 
import json
import chromadb
import voyageai
import pickle as pkl
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_voyageai import VoyageAIEmbeddings
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

/Users/malenadiazrio/Documents/UOC/TFM/TFM_2025_1/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv(override=True)
VOYAGE_API_KEY = os.getenv("VOYAGE_API_KEY")
PROJECT_DIR=os.environ["PROJECT_DIR"]

In [3]:
with open(os.path.join(PROJECT_DIR, "resources", "hpo_es.json"), "r") as fp:
    hpo = json.load(fp)

In [34]:
#Read the desired fields of the ontology
fields = ["esp_name", "esp_def", 'esp_synonyms', "esp_addons"] # "is_a",
hpo_dict = {}

for element in hpo:
    hpo_dict[element["id"]] = {field:element[field] for field in fields if field in element}

In [35]:
def_count = 0
name_count = 0
synonym_count = 0
addon_count = 0
for k,v in hpo_dict.items():
    if "esp_def" in v:
        def_count += 1
    if "esp_name" in v:
        name_count += 1
    if "esp_synonyms" in v:
        synonym_count += 1
    if "esp_addons" in v:
        addon_count += 1

print(f"""Total docs: {len(hpo_dict)}
Total elements with a spanish name: {name_count}
Total elements with a spanish definition: {def_count}
Total elements with a spanish synonym: {synonym_count}
Total elements with a spanish addon: {addon_count}
""")

Total docs: 19077
Total elements with a spanish name: 19077
Total elements with a spanish definition: 16504
Total elements with a spanish synonym: 10852
Total elements with a spanish addon: 1764



Procesar linaje

In [20]:
def clean_lineage(s):
    return s.split('!')[0].strip()

In [21]:
#clean lineage
for k,v in hpo_dict.items():
    if "is_a" in v:
        if isinstance(v["is_a"], list):
            for i, parent in enumerate(v["is_a"]):
                v["is_a"][i] = clean_lineage(parent)
        else:
            v["is_a"] = clean_lineage(v["is_a"] )

In [22]:
#clean lineage
def find_parent(hpo_code, hpo_dict=hpo_dict):
    lineage = hpo_dict[hpo_code]["is_a"]
    if isinstance(lineage, list):
        parents = set(lineage)
        for parent in lineage:
            parents.update(find_parent(parent))
        return parents
    
    if "is_a" not in hpo_dict[lineage]:
        return []
    
    return [lineage] + list(find_parent(lineage))
    

_ = {v.update({"lineage": find_parent(k)}) for k,v in hpo_dict.items() if "is_a" in v}

In [23]:
hpo_dict

{'HP:0000001': {'esp_name': 'Todos'},
 'HP:0000002': {'esp_name': 'Anomalía de la estatura',
  'esp_def': 'Desviación de la norma de estatura con respecto a la esperada según las normas de edad y sexo.',
  'esp_synonyms': 'Altura corporal anormal',
  'is_a': 'HP:0001507',
  'lineage': ['HP:0001507', 'HP:0000118']},
 'HP:0000003': {'esp_name': 'Displasia renal multiquística',
  'esp_def': 'La displasia multiquística del riñón se caracteriza por múltiples quistes de tamaño variable en el riñón y la ausencia de un sistema pelvico-iceal normal. La enfermedad se asocia a atresia ureteral o ureteropélvica, y el riñón afectado no es funcional.',
  'esp_synonyms': ['Riñón displásico multiquístico',
   'Riñones multiquísticos',
   'Displasia renal multiquística'],
  'is_a': 'HP:0000107',
  'lineage': ['HP:0000107',
   'HP:0012210',
   'HP:0000077',
   'HP:0010935',
   'HP:0000079',
   'HP:0000119',
   'HP:0000118']},
 'HP:0000005': {'esp_name': 'Modo de herencia',
  'esp_def': 'Patrón según el 

Creating info for chroma db

In [25]:
def add_to_names_dict(terms, hpo_code, names_dict):
    for term in terms:
        term = term.lower()
        if term in names_dict and hpo_code not in names_dict[term]:
            names_dict[term] += [hpo_code]
        else:
            names_dict[term] = [hpo_code]
    return names_dict

In [36]:
documents_text = []
metadata = []

for hpo_code, hpo_values in hpo_dict.items():
    for key, value in hpo_values.items():
        if isinstance(value, list):
            for item in value:
                metadata.append({'hpo_id': hpo_code, 'key':key})
                documents_text.append(item)
        else:
            metadata.append({'hpo_id': hpo_code, 'key':key})
            documents_text.append(value)            

In [ ]:
documents_text = []
metadata_list = []
names_dict = {}
for hpo_code, hpo_values in hpo_dict.items():
    metadata = {"hpo_id":hpo_code}
    cleaned_info = []
    if "esp_name" in hpo_values:
        cleaned_info.append(hpo_values["esp_name"])
        names_dict = add_to_names_dict([hpo_values["esp_name"]], hpo_code, names_dict)
    if "esp_synonyms" in hpo_values:
        syn_list = hpo_values["esp_synonyms"] if isinstance (hpo_values["esp_synonyms"], list) else [hpo_values["esp_synonyms"]]
        syn_list = [str(s) for s in syn_list]
        cleaned_info += syn_list
        names_dict = add_to_names_dict(syn_list, hpo_code, names_dict)
    if "esp_def" in hpo_values:
        cleaned_info.append(hpo_values["esp_def"])
    cleaned_info = [str(s) for s in cleaned_info]
    cleaned_info = [s.strip() + "." if not s.strip().endswith(".") else s.strip() for s in cleaned_info]
    cleaned_info = " ".join(cleaned_info)
    documents_text.append(cleaned_info)
    if "lineage" in hpo_values:
        metadata["lineage"] = "->".join(hpo_values["lineage"])
    metadata_list.append(metadata)
ids_list = [v['hpo_id'] for v in metadata_list]

Create Voyage Embeddings

In [8]:
# MODEL_NAME = "BAAI/bge-small-en-v1.5"
MODEL_NAME = "voyage-3"

# embeddings = FastEmbedEmbeddings(model_name=MODEL_NAME)
embeddings_model = VoyageAIEmbeddings(voyage_api_key=VOYAGE_API_KEY,model="voyage-3")

In [47]:
embeddings = []

In [90]:
# vo = voyageai.Client(api_key=VOYAGE_API_KEY)

# batch_size = 50
# tokens=0
# starttime = time.time()

# for i in tqdm.tqdm(range(len(embeddings), len(documents_text), batch_size), desc="Batch: " ):       
#     if tokens >= 9000:
#         while time.time() < starttime + 61:
#             time.sleep(1)
#         tokens = 0
#         starttime = time.time()

#     response= vo.embed(
#         documents_text[i:i + batch_size], model=MODEL_NAME, input_type="document"
#     )
#     tokens += response.total_tokens 
#     embeddings += response.embeddings

#     time.sleep(20)

Batch: 100%|██████████| 150/150 [51:56<00:00, 20.77s/it]


In [50]:
documents_text

['Todos',
 'Anomalía de la estatura',
 'Desviación de la norma de estatura con respecto a la esperada según las normas de edad y sexo.',
 'Altura corporal anormal',
 'Displasia renal multiquística',
 'La displasia multiquística del riñón se caracteriza por múltiples quistes de tamaño variable en el riñón y la ausencia de un sistema pelvico-iceal normal. La enfermedad se asocia a atresia ureteral o ureteropélvica, y el riñón afectado no es funcional.',
 'Riñón displásico multiquístico',
 'Riñones multiquísticos',
 'Displasia renal multiquística',
 'Modo de herencia',
 'Patrón según el cual un determinado rasgo o trastorno genético se transmite de una generación a la siguiente.',
 'Herencia',
 'Herencia autosómica dominante',
 'Modo de herencia que se observa en los rasgos relacionados con un gen codificado en uno de los autosomas (es decir, los cromosomas humanos 1-22) en el que un rasgo se manifiesta en heterocigotos. En el contexto de la genética médica, un trastorno autosómico domina

In [ ]:
idx_to_delete = [i for i,doc in enumerate(documents_text) if not isinstance(doc,str)][0]
documents_text.pop(idx_to_delete)
metadata.pop(idx_to_delete)

In [52]:
len(embeddings)

40000

In [64]:
vo = voyageai.Client(api_key=VOYAGE_API_KEY)
batch_size = 1000
# embeddings= []
for i in tqdm.tqdm(range(len(embeddings), len(documents_text), batch_size), desc="Batch: " ):       
    response= vo.embed(
        documents_text[i:i + batch_size], model=MODEL_NAME, input_type="document"
    )
    embeddings += response.embeddings


Batch: 100%|██████████| 23/23 [00:30<00:00,  1.32s/it]


In [68]:
with open( os.path.join(PROJECT_DIR, "./resources/Voyage Embeddings/embeddings_separated.pkl"), "wb") as fp:
    pkl.dump({"metadata": metadata, "docs":documents_text, "embeddings":embeddings}, fp)

In [11]:
with open(f"{PROJECT_DIR}/resources/Voyage Embeddings/embeddings_w_synonyms.pkl", "rb") as fp:
    embeddings = pkl.load(fp)

In [12]:
with open(f"{PROJECT_DIR}/resources/Voyage Embeddings/docs_w_synonyms.pkl", "rb") as fp:
    docs = pkl.load(fp)

In [24]:
ids_list =  docs['ids']
metadata2=[]
for id in ids_list:
    for m in metadata_list:
        if m['hpo_id'] == id:
            metadata2.append(m)

In [5]:
chroma_client = chromadb.HttpClient(host='localhost', port=8001)

In [76]:
metadata[0]['hpo_id']

'HP:0000001'

In [ ]:
# chroma_client = chromadb.PersistentClient(path="../../chroma_db/Voyage3")
collection = chroma_client.get_or_create_collection("hpo_ontology_esp_FULL")
ids_list = [metadata[i]['hpo_id'] + str(i) for i in list(range(len(embeddings)))]
BATCH_SIZE = 1000
for i in range(0, len(embeddings), BATCH_SIZE):
        collection.add(
                embeddings=embeddings[i: i+BATCH_SIZE],
                documents=documents_text[i: i+BATCH_SIZE],
                metadatas=metadata[i: i+BATCH_SIZE],
                ids = ids_list[i: i+BATCH_SIZE]
        )

NameError: name 'embeddings' is not defined

In [ ]:
with open("../../resources/names_dict.pkl", "wb") as fp:
    pkl.dump(names_dict, fp)

In [9]:
langchain_chroma = Chroma(
    client=chroma_client,
    collection_name="hpo_ontology_esp_FULL",
    embedding_function=embeddings_model,
)

In [10]:
print("There are", langchain_chroma._collection.count(), "documents in the collection")

There are 19077 documents in the collection


In [16]:
langchain_chroma.get_by_ids([    "HP:0002885",
    "HP:0100006",
    "HP:0011463",
    "HP:0002671",
    "HP:0002664",
    "HP:0003593"])

[Document(id='HP:0002664', metadata={'lineage': 'HP:0000118', 'hpo_id': 'HP:0002664'}, page_content='Neoplasia. Masa tisular anormal. Cáncer. Neoplasia. Anomalía oncológica. Oncología. Tumor. Tumor. Anomalía de un órgano o sistema orgánico que consiste en una proliferación celular autónoma e incontrolada que puede producirse en cualquier parte del cuerpo en forma de neoplasia (tumor) benigna o maligna.'),
 Document(id='HP:0002671', metadata={'hpo_id': 'HP:0002671', 'lineage': 'HP:0008069->HP:0000951->HP:0002664->HP:0001574->HP:0000118->HP:0011793'}, page_content='Carcinoma basocelular. Carcinomas basocelulares. Epitelioma basocelular. Nevus basocelular. Basalioma. La presencia de un carcinoma basocelular de la piel.'),
 Document(id='HP:0002885', metadata={'lineage': 'HP:0100836->HP:0100006->HP:0000707->HP:0004375->HP:0002664->HP:0012639->HP:0000118->HP:0002011->HP:0011793', 'hpo_id': 'HP:0002885'}, page_content='Meduloblastoma. Tumor embrionario de crecimiento rápido que surge en la pa

In [18]:
og = 'Hemos investigado la incidencia del síndrome de Gorlin (SG) en pacientes con el tumor cerebral infantil meduloblastoma. Se estudiaron ciento setenta y tres casos consecutivos de meduloblastoma en la North-West Regional Health Authority entre 1954 y 1989 (Manchester Regional Health Board antes de 1974). Tras revisar las notas de los casos, las radiografías y las encuestas sanitarias, sólo en 2/173 casos había pruebas que apoyaran el diagnóstico de GS. Otro caso con un 50% de riesgo de GS murió de un tumor cerebral a los 4 años de edad. Por lo tanto, la incidencia de GS en el meduloblastoma se sitúa probablemente entre el 1-2%. Para evaluar la incidencia del meduloblastoma en el GS se utilizó un estudio poblacional de GS en la región iniciado en 1983, que resultó ser de entre el 3 y el 5%. Esta cifra es inferior a las estimaciones anteriores, pero se trata del primer estudio poblacional realizado. En vista de la temprana edad de aparición del GS (media de 2 años), los niños que presentan meduloblastoma, especialmente los menores de 5 años, deben ser examinados para detectar signos del síndrome. A continuación, se identificará a aquellos con alto riesgo de desarrollar carcinomas basocelulares invasivos múltiples.'

In [29]:
sentences

['Hemos investigado la incidencia del síndrome de Gorlin (SG) en pacientes con el tumor cerebral infantil meduloblastoma',
 ' Se estudiaron ciento setenta y tres casos consecutivos de meduloblastoma en la North-West Regional Health Authority entre 1954 y 1989 (Manchester Regional Health Board antes de 1974)',
 ' Tras revisar las notas de los casos, las radiografías y las encuestas sanitarias, sólo en 2/173 casos había pruebas que apoyaran el diagnóstico de GS',
 ' Otro caso con un 50% de riesgo de GS murió de un tumor cerebral a los 4 años de edad',
 ' Por lo tanto, la incidencia de GS en el meduloblastoma se sitúa probablemente entre el 1-2%',
 ' Para evaluar la incidencia del meduloblastoma en el GS se utilizó un estudio poblacional de GS en la región iniciado en 1983, que resultó ser de entre el 3 y el 5%',
 ' Esta cifra es inferior a las estimaciones anteriores, pero se trata del primer estudio poblacional realizado',
 ' En vista de la temprana edad de aparición del GS (media de 2 

In [30]:
results = []
sentences = og.split('.')
for sentence in sentences:
    if len(sentence) > 0:
        results.append(langchain_chroma.similarity_search_with_score(sentence,k=5))
results

[[(Document(id='HP:0100312', metadata={'lineage': 'HP:0000078->HP:0000707->HP:0100835->HP:0004375->HP:0100728->HP:0007379->HP:0000119->HP:0100006->HP:0100620->HP:0010787->HP:0010785->HP:0002664->HP:0100836->HP:0012639->HP:0000118->HP:0002011->HP:0011793', 'hpo_id': 'HP:0100312'}, page_content='Germinoma cerebral. La presencia de un tumor de células germinales del cerebro.'),
   0.96852565),
  (Document(id='HP:0002885', metadata={'hpo_id': 'HP:0002885', 'lineage': 'HP:0100836->HP:0100006->HP:0000707->HP:0004375->HP:0002664->HP:0012639->HP:0000118->HP:0002011->HP:0011793'}, page_content='Meduloblastoma. Tumor embrionario de crecimiento rápido que surge en la parte posterior del vermis cerebeloso y el techo neuroepitelial del cuarto ventrículo en niños. Más raramente, el meduloblastoma surge en el cerebelo en adultos.'),
   1.0105163),
  (Document(id='HP:0010796', metadata={'hpo_id': 'HP:0010796', 'lineage': 'HP:0009733->HP:0030061->HP:0000707->HP:0004375->HP:0011792->HP:0100006->HP:00026

In [56]:
vectordb = Chroma(client=cliente_destino, embedding_function=embeddings_model, 
                  collection_name="hpo_ontology_esp_FULL")

In [ ]:
langchain_chroma.as_retriever(search_kwargs= "where_document={'$contains':'mareos'}")

In [29]:
langchain_chroma.as_retriever().invoke("Tiene dolor en el riñon izquierdo")

[Document(id='HP:0100542', metadata={'hpo_id': 'HP:0100542', 'lineage': 'HP:0012210->HP:0000077->HP:0010935->HP:0000079->HP:0000119->HP:0000118'}, page_content='Localización anormal de los riñones. Localización anormal de los riñones. Un lugar anormal del riñón.'),
 Document(id='HP:0008738', metadata={'hpo_id': 'HP:0008738', 'lineage': 'HP:0000075->HP:0000119->HP:0005217->HP:0001438->HP:0010935->HP:0000118->HP:0025031->HP:0012210->HP:0000077->HP:0000079'}, page_content='Riñón parcialmente duplicado. Riñón parcialmente duplicado. La presencia de un riñón parcialmente duplicado.'),
 Document(id='HP:0030157', metadata={'hpo_id': 'HP:0030157', 'lineage': 'HP:0012531->HP:0025142->HP:0000118'}, page_content='Dolor de costado. Dolor de costado. Dolor de riñón. Sensación desagradable caracterizada por molestias físicas (como pinchazos, palpitaciones o dolores) y que se percibe como originada en el flanco.'),
 Document(id='HP:0000085', metadata={'lineage': 'HP:0100542->HP:0012210->HP:0000077->H

BM25 Retreiver

In [34]:
from langchain_core.documents import Document

docs_list = []
for id, metadata, page_content in zip(ids_list, metadata_list, documents_text):
    docs_list.append(Document(id=id, metadata=metadata, page_content=page_content))

In [35]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

keyword_retriever = BM25Retriever.from_documents(docs_list)

In [22]:
keyword_retriever.invoke("convulsiones inducibles")

[Document(id='HP:0100601', metadata={'hpo_id': 'HP:0100601', 'lineage': 'HP:0100603->HP:0002686->HP:0032443'}, page_content='Eclampsia. Complicación aguda y potencialmente mortal del embarazo, que se caracteriza por la aparición de convulsiones tónico-clónicas, generalmente en una paciente que había desarrollado preeclampsia. La eclampsia incluye las convulsiones y el coma que se producen durante el embarazo pero que no se deben a trastornos cerebrales preexistentes u orgánicos.'),
 Document(id='HP:0032826', metadata={'hpo_id': 'HP:0032826', 'lineage': 'HP:0032825->HP:0032809->HP:0032808->HP:0032807->HP:0001250->HP:0012638->HP:0000707->HP:0000118'}, page_content='Convulsión neonatal secuencial focal. La convulsión motora secuencial electroclínica neonatal focal es un tipo de convulsión electroclínica neonatal en la que no puede detectarse la característica predominante debido a que las convulsiones se presentan con una variedad de signos focales clínicos y electrográficos, que a menudo

In [90]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectordb.as_retriever(),
                                                   keyword_retriever],
                                       weights=[0.6, 0.4])

In [150]:
with open("../../resources/keyword_retriever.pkl", 'wb') as fp:
    pkl.dump(keyword_retriever, fp)

In [4]:
with open(os.path.join(PROJECT_DIR,"./resources/keyword_retriever.pkl"), 'rb') as fp:
    keyword_retriever = pkl.load(fp)

In [ ]:
keyword_retriever.vectorizer.

Fuzzy matching

In [13]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.3 MB/s eta 0:00:00


In [27]:
from rapidfuzz import process, fuzz
from rapidfuzz.utils import default_process
# User query
query = "obesidad mórbida"

# # Build search list: [(text, phenotype_id), ...]
# search_entries = []
# ids_list2 = []
# for id, docs in zip(ids_list, documents_text):
#     for doc in docs:
#         search_entries.append(doc)
#         ids_list2.append(id)
# # Extract best match
# best_match = process.extract(query, search_entries, scorer=fuzz.QRatio, limit=10, processor=default_process)

# best_match


In [33]:
class FuzzyRetriever:
    search_entries = search_entries
    ids_list = ids_list2

    def invoke(self, query):
        results = process.extract(query, search_entries, scorer=fuzz.QRatio, limit=10, processor=default_process)
        return [(self.ids_list[result[2]], result[0], result[1]) for result in results]

In [34]:
fuzzyretriever = FuzzyRetriever()
fuzzyretriever.invoke("convulsiones inducibles")

[('HP:0007332', 'Convulsiones hemifaciales.', 79.16666666666666),
 ('HP:0007332', 'Convulsiones hemifaciales.', 79.16666666666666),
 ('HP:0007359', 'Convulsiones focales.', 79.06976744186046),
 ('HP:0007359', 'Convulsiones focales.', 79.06976744186046),
 ('HP:0002373', 'Convulsiones febriles.', 77.27272727272727),
 ('HP:0002373', 'Convulsiones febriles.', 77.27272727272727),
 ('HP:0002373', 'Convulsiones inducidas por fiebre.', 75.0),
 ('HP:0033349', 'Convulsiones crecientes.', 73.91304347826086),
 ('HP:0010819', 'Convulsiones atónicas.', 72.72727272727273),
 ('HP:0033349', 'Convulsiones en serie.', 72.72727272727273)]

In [55]:
with open("../../resources/fuzzy_retriever.pkl", 'wb') as fp:
    pkl.dump({"search_entries": search_entries, "ids":ids_list2}, fp)

In [30]:
with open(os.path.join(PROJECT_DIR, "./resources/fuzzy_retriever.pkl"), 'rb') as fp:
    fuzzyretriever= pkl.load(fp)

In [32]:
search_entries = fuzzyretriever['search_entries']
ids_list2 = fuzzyretriever['ids']